In [1]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import pybullet_envs
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from gymnasium import wrappers
from collections import deque

AttributeError: 'dict' object has no attribute 'env_specs'